In [11]:
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("content_ext_crd_cd_ln.csv")
print(data)
print(data.columns)
print(data.index)

       REPORT_ID  state_num  finance_org_num  type_dw_num  guarantee_type_num  \
0         9410.0        5.0              5.0          5.0                 5.0   
1        22999.0        7.0              7.0          7.0                 7.0   
2        25708.0        2.0              2.0          2.0                 2.0   
3        26899.0        1.0              1.0          1.0                 1.0   
4        28675.0        2.0              2.0          2.0                 2.0   
5        41549.0        5.0              5.0          5.0                 5.0   
6        48614.0        3.0              3.0          3.0                 3.0   
7        49099.0        4.0              4.0          4.0                 4.0   
8        50598.0        2.0              2.0          2.0                 2.0   
9        54038.0        1.0              1.0          1.0                 1.0   
10       70476.0       17.0             17.0         17.0                17.0   
11       74971.0        7.0 

In [3]:
# 7 个类别 *2
label =[ 'state_more', 'state_less', 'finance_org_more', 'finance_org_less', 'type_dw_more',
       'type_dw_less', 'guarantee_type_more', 'guarantee_type_less', 'payment_rating_more',
        'payment_rating_less', 'class5_state_more','class5_state_less', 'payment_state_more', 'payment_state_less']
#
cate = []
c = 0
size = 0
for i in range(len(label)):
    tmp = []
    tmp = pd.get_dummies(data[label[i]],prefix = label[i])
    size += tmp.shape[1]
    if c == 0:
        cate = tmp
        c = 3
        continue
    cate = pd.concat([cate,tmp],axis = 1)
cate.head()

,state_more_1.0,state_more_2.0,state_more_3.0,state_more_4.0,state_more_5.0,state_more_6.0,state_more_7.0,state_more_8.0,state_more_9.0,state_more_10.0,...,payment_state_less_26.0,payment_state_less_28.0,payment_state_less_29.0,payment_state_less_32.0,payment_state_less_35.0,payment_state_less_44.0,payment_state_less_50.0,payment_state_less_57.0,payment_state_less_71.0,payment_state_less_74.0
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
label = data.iloc[:,1:-14].columns
# print(label)
number = data.iloc[:,1:-14]
kk = min_max_scaler.fit_transform(number)  
numbernew = pd.DataFrame(kk,columns = label)
fulldata = pd.concat([data.REPORT_ID,numbernew,cate],axis = 1)
fulldata.head()

,REPORT_ID,state_num,finance_org_num,type_dw_num,guarantee_type_num,payment_rating_num,payment_cyc_max,payment_cyc_min,payment_cyc_mean,payment_cyc_zongshu,...,payment_state_less_26.0,payment_state_less_28.0,payment_state_less_29.0,payment_state_less_32.0,payment_state_less_35.0,payment_state_less_44.0,payment_state_less_50.0,payment_state_less_57.0,payment_state_less_71.0,payment_state_less_74.0
0,9410.0,0.010753,0.010753,0.010753,0.010753,0.010753,0.163889,0.016667,0.094150,0.005935,...,0,0,0,0,0,0,0,0,0,0
1,22999.0,0.016129,0.016129,0.016129,0.016129,0.016129,0.063889,0.033333,0.041385,0.011869,...,0,0,0,0,0,0,0,0,0,0
2,25708.0,0.002688,0.002688,0.002688,0.002688,0.002688,0.030556,0.008333,0.018106,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,26899.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.025000,0.022284,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,28675.0,0.002688,0.002688,0.002688,0.002688,0.002688,0.047222,0.027778,0.036212,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#删掉一些维度
train = pd.read_csv("../input/contest_basic_train.tsv",sep='\t')
mm = pd.concat([train.REPORT_ID,train.Y],axis = 1)
ceshidata = fulldata.merge(mm,how = 'inner',left_on = 'REPORT_ID',right_on = 'REPORT_ID') 
print(ceshidata.head())
print(ceshidata.shape)

  REPORT_ID  state_num  finance_org_num  type_dw_num  guarantee_type_num  \
0      9410   0.010753         0.010753     0.010753            0.010753   
1     22999   0.016129         0.016129     0.016129            0.016129   
2     25708   0.002688         0.002688     0.002688            0.002688   
3     26899   0.000000         0.000000     0.000000            0.000000   
4     28675   0.002688         0.002688     0.002688            0.002688   

   payment_rating_num  payment_cyc_max  payment_cyc_min  payment_cyc_mean  \
0            0.010753         0.163889         0.016667          0.094150   
1            0.016129         0.063889         0.033333          0.041385   
2            0.002688         0.030556         0.008333          0.018106   
3            0.000000         0.022222         0.025000          0.022284   
4            0.002688         0.047222         0.027778          0.036212   

   payment_cyc_zongshu ...  payment_state_less_28.0  payment_state_less_29.0  \


In [6]:
ceshidata.Y.value_counts()

0    24699
1     1535
Name: Y, dtype: int64

In [8]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = SMOTE(kind='borderline1').fit_sample(ceshidata.iloc[:,1:-1], ceshidata.iloc[:,-1])

In [18]:

print(X_resampled[0:10,:])
print(X_resampled.shape)
print(y_resampled[0:10])
print(y_resampled.shape)
pd.Series(y_resampled).value_counts()
# x_train,x_valid,y_train, y_valid= train_test_split(X_resampled,y_resampled,test_size = 0.3)
x_train,x_valid,y_train, y_valid= train_test_split(ceshidata.iloc[:,1:-1],ceshidata.iloc[:,-1],test_size = 0.3)
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

[[0.01075269 0.01075269 0.01075269 ... 0.         0.         0.        ]
 [0.01612903 0.01612903 0.01612903 ... 0.         0.         0.        ]
 [0.00268817 0.00268817 0.00268817 ... 0.         0.         0.        ]
 ...
 [0.00806452 0.00806452 0.00806452 ... 0.         0.         0.        ]
 [0.00268817 0.00268817 0.00268817 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(49398, 1620)
[0 0 0 0 1 0 0 0 0 0]
(49398,)
(18363, 1620)
(18363,)
(7871, 1620)
(7871,)


In [20]:
pd.Series(y_train).value_counts()

0    17254
1     1109
Name: Y, dtype: int64

In [21]:
# x_train = ceshidata.iloc[0:18000,1:-1]
# y_train = ceshidata.iloc[0:18000,-1]
# x_valid = ceshidata.iloc[18000:,1:-1]
# y_valid = ceshidata.iloc[18000:,-1]

skf = StratifiedKFold(n_splits=5)
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=3,
        scale_pos_weight=15,
        StratifiedKFold = True,
        seed=27)
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
# xgtest =xgb.DMatrix(dtest[predictors].values)
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=3,
    metrics='auc', early_stopping_rounds=50)
mm

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.606699,0.019804,0.697662,0.006175
1,0.625455,0.010163,0.737924,0.001027
2,0.632050,0.010838,0.755409,0.004145
3,0.635738,0.012219,0.771046,0.001033
4,0.637830,0.008932,0.779909,0.001305
5,0.637830,0.006615,0.785252,0.001929
6,0.638449,0.007111,0.791035,0.003892
7,0.636357,0.007708,0.797201,0.005924
8,0.637005,0.009252,0.801142,0.005946
9,0.639055,0.008565,0.805955,0.005393


In [24]:

print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(18363, 1620)
(18363,)
(7871, 1620)
(7871,)


In [56]:
feature = 0
for i in range(5):
    xgb2 = XGBClassifier(
            learning_rate =0.1,
            n_estimators=mm.shape[0],
            max_depth=5,
            min_child_weight=1,
            gamma=0,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,   #15:1
            StratifiedKFold = True,
            seed=27)
    xgb2.fit(x_train,y_train)
    feature += xgb2.feature_importances_
    print(i)
#     pred = xgb2.predict_proba(x_valid)[:,1]
#     print(pred)
#     auc = roc_auc_score(y_valid, pred)  
#     auc
feature = feature/5

0
1
2
3
4


In [66]:
print(x_train.columns)
feature[feature>0.0000008]
x_train.columns[feature>0.0000008]

Index(['state_num', 'finance_org_num', 'type_dw_num', 'guarantee_type_num',
       'payment_rating_num', 'payment_cyc_max', 'payment_cyc_min',
       'payment_cyc_mean', 'payment_cyc_zongshu', 'payment_cyc_num',
       ...
       'payment_state_less_26.0', 'payment_state_less_28.0',
       'payment_state_less_29.0', 'payment_state_less_32.0',
       'payment_state_less_35.0', 'payment_state_less_44.0',
       'payment_state_less_50.0', 'payment_state_less_57.0',
       'payment_state_less_71.0', 'payment_state_less_74.0'],
      dtype='object', length=1620)


Index(['state_num', 'payment_cyc_max', 'payment_cyc_min', 'payment_cyc_mean',
       'payment_cyc_zongshu', 'credit_limit_amount_max',
       'credit_limit_amount_min', 'credit_limit_amount_mean',
       'credit_limit_amount_zongshu', 'balance_max',
       ...
       'payment_state_more_2.0', 'payment_state_more_4.0',
       'payment_state_more_6.0', 'payment_state_more_9.0',
       'payment_state_more_10.0', 'payment_state_more_13.0',
       'payment_state_more_17.0', 'payment_state_less_1.0',
       'payment_state_less_2.0', 'payment_state_less_6.0'],
      dtype='object', length=148)

In [70]:
fulldata.head()
fulldataok = []
fulldataok = fulldata[x_train.columns[feature>0.0000008]]  #一定要小心是谁的ID
fulldataok = pd.concat([fulldata.REPORT_ID,fulldataok],axis = 1)
fulldataok.head()

,REPORT_ID,state_num,payment_cyc_max,payment_cyc_min,payment_cyc_mean,payment_cyc_zongshu,credit_limit_amount_max,credit_limit_amount_min,credit_limit_amount_mean,credit_limit_amount_zongshu,...,payment_state_more_2.0,payment_state_more_4.0,payment_state_more_6.0,payment_state_more_9.0,payment_state_more_10.0,payment_state_more_13.0,payment_state_more_17.0,payment_state_less_1.0,payment_state_less_2.0,payment_state_less_6.0
0,9410.0,0.010753,0.163889,0.016667,0.094150,0.005935,0.005185,0.000400,0.005099,0.000000,...,0,0,0,0,0,0,0,1,0,0
1,22999.0,0.016129,0.063889,0.033333,0.041385,0.011869,0.004630,0.000880,0.003150,0.005291,...,0,1,0,0,0,0,0,1,0,0
2,25708.0,0.002688,0.030556,0.008333,0.018106,0.000000,0.000306,0.000200,0.000397,0.000000,...,0,0,0,0,0,0,0,1,0,0
3,26899.0,0.000000,0.022222,0.025000,0.022284,0.000000,0.000685,0.001480,0.001367,0.000000,...,0,0,0,0,0,0,0,1,0,0
4,28675.0,0.002688,0.047222,0.027778,0.036212,0.000000,0.000463,0.000978,0.000913,0.000000,...,0,0,0,0,0,0,0,1,0,0


In [72]:
fulldataok.to_csv("../add/content_ext_crd_cd_lnok.csv",index = False)